# Outils de création d'une carte interactive par catégories

La récupération des types de lieux permet de trier les lieux par catégories et de créer une carte interactive en fonction du classement créé. 

La carte est enregistrée en format html.

Il est possible à la fin d'ajouter, à la fin du notebook, son propre fond de carte.



In [ ]:
#@markdown # Connexion à son compte GoogleDrive:

#@markdown- Lancer la cellule
#@markdown- Cliquer sur le lien généré en dessous
#@markdown Copier le mot de passe dans la nouvelle fenêtre ouverte où l'accès au compte est requis et le coller en dessous de la cellule.


!pip install geopy==1.17.0
from google.colab import drive
import urllib.parse, urllib.request
import os
import folium
from collections import Counter
from geopy.geocoders import Nominatim
import pandas as pd
from folium import plugins
import json
import csv
import re
import itertools
import matplotlib.pyplot as plt
from pandas.plotting import table

drive.mount('/content/drive/')
%cd /content/drive/My Drive/

In [ ]:
#@markdown # Outil de création d'une carte représentant les lieux par types de lieux

#@markdown Les coordonnées GPS sont récupérées pour intégrer directement les lieux sur la carte.

#@markdown La carte est originallement cadrée sur l'Europe. Il est possbile de zoomer / dézoomer et se déplacer pour accéder à l'endroit précis recherché.


nom_fichier_donnees_lieux = 'donnees lieux corpus guides.json'#@param {type:"string"}
nom_carte = 'carte_categorisee.html'#@param {type:"string"}

liste_des_types = []
os.chdir('/content/drive/My Drive/tutoriel_extraction_cartographie-main/')
with open(nom_fichier_donnees_lieux, 'r') as po:
  contenu_json = json.load(po)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_types=value["Type de lieu"]
      liste_des_types.append(test_types)

for ele in liste_des_types:
  if type(ele) == list:
    liste_des_types.remove(ele)
items = Counter(liste_des_types).keys() 
nbr_types_lieux = len(items)

### Affilier un numéro de groupe pour chaque catégorie
groupe = []
d = {ni: indi for indi, ni in enumerate(set(liste_des_types))}
numbers = [d[ni] for ni in liste_des_types]

for k, v in zip(contenu_json, numbers):
  groupe.append(v)
  for element in groupe:
    contenu_json[k]["Groupe"] = element

### Création du fonds de carte
map = folium.Map(location=[48.856614, 2.3522219], zoom_start=12)
fg = folium.FeatureGroup(name="Tous")
map.add_child(fg)

## Création des différentes catégories pour la carte
def creating_categories(nombre_categories, liste_noms_categories):
  g = []
  for k in contenu_json: 
    lat = contenu_json[k]["Latitude"] 
    longi = contenu_json[k]["Longitude"]
    for i, j in zip(range(nombre_categories), liste_noms_categories): 
      g=plugins.FeatureGroupSubGroup(fg, str(j)) 
      try:
        if str(j) == str(contenu_json[k]["Type de lieu"]):  
          try:
            folium.Marker([float(lat), float(longi)]).add_to(g) 
            map.add_child(g)  
          except (TypeError, ValueError) as Te:
            pass
      except KeyError as kerr:
        pass

creating_categories(nbr_types_lieux, liste_des_types)

folium.LayerControl(collapsed=False).add_to(map)

map.save(nom_carte)
map


In [ ]:
### Création du fonds de carte


nom_fichier_donnees_lieux = 'donnees lieux corpus guides.json'#@param {type:"string"}
lien_tiles_pour_fond_carte = "https://mapwarper.net/maps/tile/54525/{z}/{x}/{y}.png"#@param {type:"string"}
nom_carte_fond = 'carte_categorisee_fond.html'#@param {type:"string"}
map = folium.Map(location=[48.856614, 2.3522219], zoom_start=12, 
                 tiles=lien_tiles_pour_fond_carte, attr="None")



fg = folium.FeatureGroup(name="Tous")

map.add_child(fg)

liste_des_types = []
os.chdir('/content/drive/My Drive/tutoriel_extraction_cartographie-main/')
with open(nom_fichier_donnees_lieux, 'r') as po:
  contenu_json = json.load(po)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_types=value["Type de lieu"]
      liste_des_types.append(test_types)

for ele in liste_des_types:
  if type(ele) == list:
    liste_des_types.remove(ele)
items = Counter(liste_des_types).keys() 
nbr_types_lieux = len(items)

### Affilier un numéro de groupe pour chaque catégorie
groupe = []
d = {ni: indi for indi, ni in enumerate(set(liste_des_types))}
numbers = [d[ni] for ni in liste_des_types]

for k, v in zip(contenu_json, numbers):
  groupe.append(v)
  for element in groupe:
    contenu_json[k]["Groupe"] = element

## Création des différentes catégories pour la carte
def creating_categories(nombre_categories, liste_noms_categories):
  g = []
  for k in contenu_json: ### boucle: pour chasue lieu dontenu dans "donnees"
    lat = contenu_json[k]["Latitude"] 
    longi = contenu_json[k]["Longitude"]
    for i, j in zip(range(nombre_categories), liste_noms_categories): 
      g=plugins.FeatureGroupSubGroup(fg, str(j)) 
      try:
        if str(j) == str(contenu_json[k]["Type de lieu"]):  
          try:
            folium.Marker([float(lat), float(longi)]).add_to(g) 
            map.add_child(g)  
          except (TypeError, ValueError) as Te:
            pass
      except KeyError as kerr:
        pass

creating_categories(nbr_types_lieux, liste_des_types)

folium.LayerControl(collapsed=False).add_to(map)

map.save(nom_carte_fond)
map
